# Preamble

This notebook aims to illustrate some feature importance methods using the [Heart Failure Prediction dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data). **Feature importance is defined as how much of an impact a feature has on a model's predictions.** The dataset we'll use has 12 numerical features for each example and the target for each feature is whether or not death occurred (binary classification). The topics covered and coded are as follows:

- Feature value histograms
- Permutation importance
- SHAP summary plots
- SHAP dependence contribution plots

*Note: this notebook is a follow-up to one of my other notebooks [Feature selection for heart disease prediction](https://www.kaggle.com/valbauman/feature-selection-for-heart-disease-prediction). Big thanks to the Kaggle course Machine Learning Explainability where I learned about some of the topics used here.*

# Create + Train SVM model
In my previous notebook, we trained an SVM model that used 5 of the 12 features to predict whether or not death occurred. We will use this same model to explain the importance of each feature used in this model.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score

#load data and create 90/10 train/validation splits
data= pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv', delimiter= ',')
top5_feats= ['age', 'ejection_fraction', 'serum_creatinine', 'serum_sodium', 'time']
feats= data.loc[:,top5_feats] #only keep 5 of the 12 feature columns
labels= data.iloc[:,-1]
x_train, x_devel, y_train, y_devel= train_test_split(feats, labels, test_size= 0.1, random_state= 20)

#scale features (0 mean, unit variance for each)
scaler= StandardScaler()
x_train_scale= scaler.fit_transform(x_train)
x_devel_scale= scaler.transform(x_devel)

#train SVM
svm= SVC(probability= True)
svm.fit(x_train_scale, y_train)

#for interest's sake, we'll display the average recall of each class for this model
predicts= svm.predict(x_devel_scale)
print(balanced_accuracy_score(y_devel, predicts))

Obviously this score is not great but the focus of this notebook is **feature importance**, not trying to get the best prediction accuracies.

# Feature Value Histograms
For each class, we can visualize the values for each feature using histograms.

In [ ]:
def create_hist(feature_vals, labels):
    """
    FUNCTION TO DISPLAY HISTOGRAMS OF THE FEATURE VALUES FOR THE TWO CLASSES
    
    INPUTS:
    feature_vals : pd series of the values of the single feature of interest
    labels : pd series of the corresponding labels for the feature values
    
    OUTPUTS:
    None (displays histograms only)
    """
    feat_0= feature_vals.loc[y_train == 0] #feature values that belong to class 0
    feat_1= feature_vals.loc[y_train == 1] #features values that belong to class 1
    
    #for histogram bins, get the min and max feature values. we'll use a total of 10 bins using the min/max
    feat_min= np.min(feature_vals)
    feat_max= np.max(feature_vals)
    bins= np.linspace(feat_min, feat_max, num= 10)
    
    #display histograms. illustrates the distribution of values for the feature for each class
    plt.figure()
    plt.hist(feat_0, bins, alpha= 0.5, label= 'Target= 0')
    plt.hist(feat_1, bins, alpha= 0.5, label= 'Target= 1')
    plt.legend()
    plt.xlabel('Feature value'); plt.ylabel('Frequency')
    plt.title(('Feature value histograms - '+ feature_vals.name))
    
    return

for i in top5_feats:
    create_hist(x_train[i], y_train)

**From these histograms, we can see that both classes take on the same or similar values for most of the features.** The fact that there's a lot of overlap may explain our model's mediocre performance - since there's a lot of overlap in feature values across the two classes, the model may have difficulty distinguishing between classes by using only these features. If we did some feature engineering and added some interaction features, for example, that did not have much or any overlap, our model's average recall may have been better. Although we haven't quantified the importance of each feature using these histograms, it's interesting to visualize the values of each feature for the two classes being considered.

# Permutation Importance
Permutation importance is one approach to quantifying feature importance. It involves shuffling the values for a single feature so they no longer correspond to their original observation and then making a prediction using these shuffled feature values. We repeat this for each feature for a set number of times so that we can get a mean and standard deviation importance for each feature. The worse a prediction is with the feature values shuffled, the higher the feature's permutation importance. We'll use our trained SVM model and the five features to illustrate how to get permutation importance.

In [ ]:
from sklearn.inspection import permutation_importance

#express scaled development set as a dataframe before getting importances
x_devel_scaledf= pd.DataFrame(x_devel_scale, columns= x_devel.columns, index= x_devel.index)

permut_importance= permutation_importance(svm, x_devel_scaledf, y_devel, n_repeats= 5, random_state= 20)
print(x_devel_scaledf.columns)
print(permut_importance['importances_mean'].round(3))
print(permut_importance['importances_std'].round(3))

***Time* (follow-up period in days) has the highest importance of the five features followed by *serum_sodium* and *age*. The mean importance of *time*, *serum_sodium*, and *age* are 0.087, 0.013, and 0.02 respectively.** The negative signs on the importances for ejection_fraction and serum_creatinine  signify that those features are entirely unimportant - random chance in the shuffling made the predictions on the shuffled data more accurate than without shuffling.

# SHAP Summary Plots
Although permutation importance values are great for quantitively comparing features, they don't tell us *how* each feature matters - this is where SHAP (Shapley Additive Explanations) summary plots are helpful. The goal of SHAP is to explain a prediction from any "black box" machine learning model as a sum of contributions from it's individual feature values. The individual feature values can be thought of as players in a cooperative game where the payout is the model's prediction.

Without delving too heavily into the theory behind calculating SHAP values,we can train a number of interpretable weighted linear regression models using the predictions of our black box model (SVM) and the regression coefficients of these surrogate linear regression models give us the approximate SHAP value for each feature. This is all accomplished using KernelExplainer from the shap library.

In [ ]:
import shap

#explain the model and then get SHAP values
explain= shap.KernelExplainer(svm.predict_proba, x_train_scale, link= 'logit')
shap_vals= explain.shap_values(x_devel_scale, nsamples= len(x_devel_scale), l1_reg= 'num_features(5)')

#get shap summary plot
shap.summary_plot(shap_vals, x_devel_scaledf)

**From this plot, the feature *time* had the greatest influence on the predictions for both classes followed by *ejection_fraction* and then *serum_creatinine*.** This is indicated by the size of the bars for each class for each feature (for both classes, the bars were largest for the feature *time*).

For each of the two target classes, we can create a density scatter of SHAP values. The plot below has the values for class 0.

In [ ]:
shap.summary_plot(shap_vals[0],x_devel_scaledf)

**From this plot, we can see that high values for *time* and low values for *serum_creatinine* increases the probability that an example is classified as 0 (no death event).** High values for *time* (indicated by the red colouring) and low values for *serum_creatinine* (indicated by the blue colouring) have high SHAP values (x-axis of plot).

# SHAP Dependence Plots
Using the same SHAP values calculated previously, SHAP dependence plots reveal interaction effects between features.

In [ ]:
shap.dependence_plot('time', shap_vals[0], x_devel_scaledf, interaction_index= 'ejection_fraction')

From the shape of this plot, we can see that the larger the feature value *time*, the higher that model's probability is for predicting the label. Based on the spread of this plot, other features must interact with this feature to predict the label. We can also see that there is no linear correlation between *time* and *ejection_fraction* based on the combination of the colouring and positioning of the points.

# Conclusions
In this notebook, we visualized the feature values from a binary classification dataset using histograms to qualitatively decide if there are similar values for some features across classes. We also used a model trained on a 5-feature feature set to quantify the importance of each feature on making a prediction. The SHAP summary plots used in this notebook illustrated how each feature matters in making a prediction.

The 5 features used were *time*, *ejection_fraction*, *serum_creatinine*, *age*, and *serum_sodium*. *Time* had the highest permutation importance and was therefore the most important feature. The SHAP summary plots indicated that high values for *time* and low values for *serum_creatinine* increase the probability that an example is classified as 0 (non-death event).